In [12]:
import pandas as pd
from selenium import webdriver
import os
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import urllib3
http = urllib3.PoolManager()
from time import sleep
import re
import os.path


chromdriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromdriver)

In [13]:
def scraper():
    url = getURL()  #get the url
    folderName = getFolderName()
    pages = getPages()
    driver.get(url)   #manually log in
    releaseFile(url, pages, folderName)

In [16]:
def getURL():
    url = input('Enter the url:')
    return url


def getFolderName():
    folderName = input('create a folder (with school name) and enter the folder name:')
    return folderName
    

def getPages():
    pages = input('Enter how many pages you want to scrper: ')
    return eval(pages)
    
    
def releaseFile(url, pages, folderName):
    image_urls = []
    names = []

    pagenums = 0

    i=0
    while(i < pages):
        delay = 3 # seconds
        try:
            myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'ember47')))
            print ("Page is ready!")
        except TimeoutException:
            print ("Loading took too much time!")

        driver_html = driver.page_source
        soup = BeautifulSoup(driver_html, features="html.parser")

        main_table = soup.find_all("div", 
                               {"class": "blended-srp-results-js pt0 pb4 ph0 container-with-shadow"})

        rows = main_table[0].find_all("div",
                                 {"class": "search-entity search-result search-result--person search-result--occlusion-enabled ember-view"})


        for r in rows:
            images = r.find_all('img')
            try: name = r.find_all("span", {"class" : "name actor-name"})
            except: name = ["anonymous"]
            for image in images:
                image_urls.append(image['src'])
                try: names.append(name[0].text)
                except: names.append('anonymous')

        if i==0:
            pagenums = 2
            driver.get(url+"&page=2")
        else:
            try:
                driver.get(url+"&page=" + str(pagenums))
            except:
                break

        pagenums += 1
        i += 1   
    

    url = image_urls[0]

    for i, url in enumerate(image_urls):
        sleep (0.25)
        connection_pool = urllib3.PoolManager()
        resp = connection_pool.request('GET',url )
        url_name = url.split("=")[-1]
        fname = names[i].strip()
        fname = re.sub("['\"]","",fname)
        fname = fname + "_a_" + url_name
        f = open(folderName + '/' + fname + '.jpg', 'wb')
        f.write(resp.data)
        f.close()
        resp.release_conn()

In [ ]:
scraper()

Enter the url:https://www.linkedin.com/search/results/all/?keywords=zhejiang%20normal%20university&origin=GLOBAL_SEARCH_HEADER
create a folder (with school name) and enter the folder name:images_ZJNU
Enter how many pages you want to scrper: 3
